In [1]:
# Let's write your code here!

from flask import Flask, request, abort
from flask_basicauth import BasicAuth
import json
import pymysql
import os
import math

app = Flask(__name__)
app.config.from_file("flask_config.json", load=json.load)
app.config['BASIC_AUTH_FORCE']=True       # Reinforces the security to ask the login and password each time
auth = BasicAuth(app)




In [2]:
@app.route("/movies/<int:movie_id>")
@auth.required
def movie(movie_id):
    """the overall function"""

    def remove_null_fields(obj):
        """The function to remove the null fields in the 'columns of sql"""
        return {k:v for k, v in obj.items() if v is not None}

    db_conn = pymysql.connect(host="localhost", user="root", password = "Alsimar10", database="bechdel",
                              cursorclass=pymysql.cursors.DictCursor)
    with db_conn.cursor() as cursor:   # for movies, changing the names and keeping only the columns we want
        cursor.execute("""SELECT
                            M.movieId,
                            M.originalTitle,
                            M.primaryTitle AS englishTitle,
                            B.rating AS bechdelScore,
                            M.runtimeMinutes,
                            M.startYear AS Year,
                            M.movieType,
                            M.isAdult,
                            case 
                                when B.rating > 2 then 'True'
                                else 'False'
                            end as bechdelTest
                        FROM Movies M
                        JOIN Bechdel B ON B.movieId = M.movieId 
                        WHERE M.movieId=%s""", (movie_id, ))
        movie = cursor.fetchone()
        if not movie:
            abort (404)
        movie = remove_null_fields(movie)  # removing the null values using the function defined above

    with db_conn.cursor() as cursor: # for genre
        cursor.execute("""SELECT * FROM MoviesGenres WHERE movieId=%s""", (movie_id, ))
        genres = cursor.fetchall()
        movie['genres'] = [g['genre'] for g in genres]

    with db_conn.cursor() as cursor:  # for people
        cursor.execute("""SELECT
                            P.personId,
                            P.primaryName AS name,
                            P.birthYear,
                            P.deathYear,
                            MP.job,
                            MP.category AS role
                        FROM MoviesPeople MP
                        JOIN People P on P.personId = MP.personId
                        WHERE MP.movieId=%s""", (movie_id, ))
        people = cursor.fetchall()
        movie['people'] = [remove_null_fields(p) for p in people]
    

    db_conn.close() 

    #movie['bechdelTest']=movie['bechdelScore'].apply(lambda x : True if int(x)>0 else False) # my attempt of doing the boolean test via python

    return movie


In [3]:

PAGE_SIZE = 30  # initialising my variables
MAX_PAGE_SIZE = 30

@app.route("/movies")
@auth.required
def movies():
    """the overall function for the movies per page"""

    def remove_null_fields(obj):
        """The function to remove the null fields in the 'columns of sql"""
        return {k:v for k, v in obj.items() if v is not None}

    page = int(request.args.get('page',0))
    page_size = int(request.args.get('page_size', MAX_PAGE_SIZE))
    page_size = min(page_size, MAX_PAGE_SIZE)

    db_conn = pymysql.connect(host="localhost", user="root", password = "Alsimar10", database="bechdel",
                              cursorclass=pymysql.cursors.DictCursor)

    with db_conn.cursor() as cursor:   # for movies, changing the names and keeping only the columns we want
        cursor.execute("""SELECT
                            M.movieId,
                            M.originalTitle,
                            M.primaryTitle AS englishTitle,
                            B.rating AS bechdelScore,
                            M.runtimeMinutes,
                            M.startYear AS Year,
                            M.movieType,
                            M.isAdult,
                            case 
                                when B.rating > 2 then 'True'
                                else 'False'
                            end as bechdelTest
                        FROM Movies M
                        JOIN Bechdel B ON B.movieId = M.movieId 
                        ORDER BY movieId
                        LIMIT %s
                        OFFSET %s
                       """, (PAGE_SIZE, page * PAGE_SIZE))
        movies = cursor.fetchall()
        if not movies:
            abort (404)
        # movies = remove_null_fields(movies)  # removing the null values using the function defined above
        movie_ids = list(movies['movies']['movieId'])

    with db_conn.cursor() as cursor:
        cursor.execute("SELECT COUNT(*) AS total FROM Movies")
        total = cursor.fetchone()
        last_page = math.ceil(total['total'] / page_size)

    db_conn.close() 

    return {'movies': movies, 
            'next_page': f'/movies?page={page+1}&page_size={page_size}',
              'last_page': f'/movies?page={last_page}&page_size={page_size}',
            }

In [13]:
big_movie= {
    "last page": "value1",
    "movies": [{
        "movieId": "subvalue1",
        "subkey2": "subvalue1bis"
    }, {
        "movieId": "subvalue2",
        "subkey2": "subvalue2bis"
    }],
    "next_page": "value3"
}

print(big_movie)


{'last page': 'value1', 'movies': [{'movieId': 'subvalue1', 'subkey2': 'subvalue1bis'}, {'movieId': 'subvalue2', 'subkey2': 'subvalue2bis'}], 'next_page': 'value3'}


In [18]:
len(big_movie['movies'])
big_movie['movies']

[{'movieId': 'subvalue1', 'subkey2': 'subvalue1bis'},
 {'movieId': 'subvalue2', 'subkey2': 'subvalue2bis'}]

In [26]:
list_movieId = []
for m in big_movie['movies']:
    test = m['movieId']
    list_movieId.append(test)
list_movieId 

['subvalue1', 'subvalue2']

In [ ]:
with db_conn.cursor() as cursor:
    # SQL query with parameterized IN clause
    sql = "SELECT * FROM your_table_name WHERE client_id IN (%s)"
            
    # Generate placeholders for client_ids based on the length of the list
    placeholders = ','.join(['%s'] * len(client_ids))
            
    # Format the SQL query with the placeholders
    sql = sql % placeholders
            
    # Execute the SQL query with client_ids as parameters
    cursor.execute(sql, tuple(client_ids))
            
    # Fetch the query results
    result = cursor.fetchall()
            
    return result
            
finally:
        # Close the database connection
        db_conn.close()

# Example usage:
client_ids = [1001, 1002, 1003]  # List of client IDs
data_for_clients = select_data_for_clients(client_ids)
print(data_for_clients)


In [27]:
!pip install flask_swagger_ui

  Obtaining dependency information for flask_swagger_ui from https://files.pythonhosted.org/packages/a5/20/df137e8efb744209135f70cb2b00c8917369ae4177370d1fab1789a0675c/flask_swagger_ui-4.11.1-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/1.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.4 MB ? eta -:--:--
   ---- ----------------------------------- 0.1/1.4 MB 1.7 MB/s eta 0:00:01
   ------------------------------------- -- 1.3/1.4 MB 10.2 MB/s eta 0:00:01
   ---------------------------------------- 1.4/1.4 MB 9.8 MB/s eta 0:00:00
